In [1]:
import requests
import urllib
import os
import pandas as pd
from parseTable import *
rawdataDir=os.path.join(os.getcwd(),"rawdata")
url_swissgdpquandl="https://www.quandl.com/api/v3/datasets/FRED/GDP.csv?api_key=2xGzWaXGnzUU5eBa58m4"

url_Nasdaq_snb_hold="https://www.nasdaq.com/quotes/institutional-portfolio/swiss-national-bank-913041?page={0}"

url="https://www.nasdaq.com/quotes/institutional-portfolio/swiss-national-bank-913041"
urlnextpage="https://www.nasdaq.com/quotes/institutional-portfolio/swiss-national-bank-913041?page=2"
urlincreased="https://www.nasdaq.com/quotes/institutional-portfolio/swiss-national-bank-913041/increased"
urlincreasedPage2="https://www.nasdaq.com/quotes/institutional-portfolio/swiss-national-bank-913041/increased?page=2"
urldecreased="https://www.nasdaq.com/quotes/institutional-portfolio/swiss-national-bank-913041/decreased"

In [2]:
if not os.path.exists(rawdataDir):
    os.makedirs(rawdataDir)
nasdaq_snb_held=pd.DataFrame()

In [ ]:
page=5

for p in range(0,page):
    url = url_Nasdaq_snb_hold.format(p+1)
    r = requests.get(url, allow_redirects=True)
    if(r.status_code!=200):
        print ("Cannot fresh page {0}".format(url))
    else:
        page = r.text
        print ("page successfully read {0}".format(url))
    from bs4 import BeautifulSoup
    soup = BeautifulSoup(page, 'html.parser')
    swisspage=soup.find("div", {"id": "total-positions-table"}).table
    df=parse_html_table(swisspage)
    nasdaq_snb_held=nasdaq_snb_held.append(df,ignore_index=True)

page successfully read https://www.nasdaq.com/quotes/institutional-portfolio/swiss-national-bank-913041?page=1
page successfully read https://www.nasdaq.com/quotes/institutional-portfolio/swiss-national-bank-913041?page=2
page successfully read https://www.nasdaq.com/quotes/institutional-portfolio/swiss-national-bank-913041?page=3


In [ ]:
from datetime import datetime
asoftxt=soup.find("div", {"id": "institution-position-stats"}).h2.span.text
asoftxt=datetime.strptime(asoftxt,'%m/%d/%Y').strftime("%Y%m%d")

nasdaq_snb_held.to_csv(os.path.join(rawdataDir,"nasdaq_snb_held{0}.csv".format(asoftxt)),index=False)

# Get Swiss GDP

In [ ]:
r = requests.get(url_swissgdpquandl, allow_redirects=True)

In [ ]:
swissgdpRawFile=os.path.join(rawdataDir,"swissgdp.csv")
open(swissgdpRawFile, 'bw').write(r.content)
swisshistoricalgdp= pd.read_csv( swissgdpRawFile)

In [ ]:
swisshistoricalgdp["Date"]=pd.to_datetime (swisshistoricalgdp["Date"],format="%Y-%m-%d")

In [ ]:
swisshistoricalgdp.columns